# Remote Controlled Operation

This notebook cuts the middle man from the teleoperation. Instead of using your workstation to relay controller input to the robot, plug the controller's dongle into a USB port on the the Jetbot itself.

Since the controller will not be picked up by Linux, we will use [pyusb](https://github.com/pyusb/pyusb) to control the Jetbot. The code in this notebook is based on Kees Jan Koster's [Python code for ShanWan PS3-style Controller](https://github.com/kjkoster/shanwan-ps3-controller-clone).


---
## Install Libraries and do Imports
Since the controller is not natively supported, we use direct USB commands to read the buttons on the controller. For that we need `pyusb`, which does not come preinstalled in Jetpack.

In [ ]:
!pip3 install pyusb


In [ ]:
import usb.core
import usb.util
from time import sleep
from jetbot import Robot


---
## Define USB Device Connection
The control loop needs a connection endpoint. Here we search the USB bus for the controller, reset it, detach any associated processes and then claim it for ourselves. We expect the controller to have vendor id `0x2563` and product id `0x0526`. You can verify that your controller has the same by running `lsusb` and reviewing the output.

The gamepad buttons can then be polled on endpoint address `0x81`, which is equivalent to configuration 0, interface 0 and endpoint 0 on the device. We could take the short route and just query for that endpoint, but I chose to follow the official USB device descriptor structure. Most examples you see take shortcuts here.

In [ ]:
idVendor = 0x2563
idProduct = 0x0526
configuration_index = 0
interface_index = 0
endpoint_index = 0

controller = usb.core.find(idVendor=idVendor, idProduct=idProduct)
if controller is None:
    print("no controller detected")
else:
    if controller.is_kernel_driver_active(interface_index):
        controller.detach_kernel_driver(interface_index)
    usb.util.claim_interface(controller, interface_index)
    interface = controller[configuration_index].interfaces()[interface_index]
    endpoint = interface.endpoints()[endpoint_index]
    print(f"reading from endpoint 0x{endpoint.bEndpointAddress:02x} on {controller.manufacturer} {controller.product} at {idVendor:04x}:{idProduct:04x} on /dev/bus/usb/{controller.bus:03d}/{controller.address:03d}")
    

Next we define how to map the analog joystick values from the right analog joystick.

The X-axis value from the joystick is simply scales from the `0..255` range of the analog joystick to the motor control range `-1.0..1.0`. This is fairly aggressive and you'll have to be a little light on the throttle to avoid your robot skidding.

The Y-axis is scaled too, but into a rotation factor. For turning, we do not make use of the full motor speed range. Using the full range makes it really hard to elegantly turn the robot. Instead, we map to `-0.4..0.4` as the control range.


In [ ]:
def scale(val, src, dst):
    return ((val - src[0]) / (src[1]-src[0])) * (dst[1]-dst[0]) + dst[0]
    
def map(joystick_x, joystick_y):
    # first scale forward/backward motion based on the Y-axis value
    y_speed = scale(joystick_y, (0, 255), (1.0, -1.0))
    
    # then add the left/right turning based on the X-axis value
    left_rotation = scale(joystick_x, (0, 255), (-0.4, 0.4))
    
    # the actual motor value is the sum of the two
    return left_rotation + y_speed, -left_rotation + y_speed
    

---
# Robot Connection
The robot connection is a lot simpler than the controller code. Just make the object and we're good to go.

In [ ]:
robot = Robot()


---
# Main Control Loop
With everything detected, claimed and set up, it os now time to start the main control loop. This is not very fancy, we just poll the controller for button states and set the motor states accordingly.

The analog values for the right joystick can be found in `packet[3]` for the X-axis and `packet[4]` for the Y-axis. If you prefer to use the left joystick, you can change these to `packet[1]` and `packet[2]` respectively.


In [ ]:

while True:
    packet = endpoint.read(endpoint.wMaxPacketSize)
    left, right = map(packet[3], packet[4])
    robot.set_motors(left, right)


Alternatively, you can use the D-pad for control. These can be found in `packet[5]`. Not sure what the history on those specific values is, but the D-pad does not use a bitmap.

Here too, we don't use the full control range of the motors, as that is just too aggressive.


In [ ]:
while True:
    packet = endpoint.read(endpoint.wMaxPacketSize)
    if   packet[5] == 0b0000:
        robot.set_motors(0.5, 0.5)
    elif packet[5] == 0b0110:
        robot.set_motors(0.0, 0.5)
    elif packet[5] == 0b0010:
        robot.set_motors(0.5, 0.0)
    elif packet[5] == 0b0100:
        robot.set_motors(-0.5, -0.5)
    else:
        robot.set_motors(0.0, 0.0)
